In [1]:
# Step 1: Install Required Libraries
!pip install streamlit pyngrok pandas matplotlib openpyxl

# Step 2: Write the Streamlit App to a File
with open('streamlit_app.py', 'w') as f:
    f.write("""
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt

# Title of the App
st.title("Company Financial Dashboard")

# Step 3: File Uploader
uploaded_file = st.file_uploader("Upload your dataset (Excel file)", type=["xlsx"])

if uploaded_file:
    # Load the uploaded file
    data = pd.read_excel(uploaded_file)
    data['int payment'] = data['int payment'].replace(',', '', regex=True).astype(float)
    data['out payment'] = data['out payment'].replace(',', '', regex=True).astype(float)

    # Immediate and Future Transactions
    immediate_transactions = data[data['Type_K'].isin(['IN PAYMENTS', 'OUT PAYMENTS'])]
    future_transactions = data[data['Type_K'].isin(['SALE', 'PURCHASE'])]

    # KPI 1: Total Immediate In and Out Payments
    st.subheader("KPI 1: Total Immediate In and Out Payments")
    total_immediate_in = immediate_transactions['int payment'].sum()
    total_immediate_out = immediate_transactions['out payment'].sum()
    fig, ax = plt.subplots(figsize=(8, 5))
    ax.bar(['Immediate In', 'Immediate Out'], [total_immediate_in, total_immediate_out], color=['blue', 'orange'])
    ax.set_title('Immediate In vs Out Payments')
    ax.set_ylabel('Total Payments')
    st.pyplot(fig)

    # KPI 2: Total Future In and Out Payments
    st.subheader("KPI 2: Total Future In and Out Payments")
    total_future_in = future_transactions['int payment'].sum()
    total_future_out = future_transactions['out payment'].sum()
    fig, ax = plt.subplots(figsize=(8, 5))
    ax.bar(['Future In', 'Future Out'], [total_future_in, total_future_out], color=['green', 'red'])
    ax.set_title('Future In vs Out Payments')
    ax.set_ylabel('Total Payments')
    st.pyplot(fig)

    # KPI 3: Net Payment Balances
    st.subheader("KPI 3: Net Payment Balances")
    net_immediate = total_immediate_in - total_immediate_out
    net_future = total_future_in - total_future_out
    fig, ax = plt.subplots(figsize=(8, 5))
    ax.bar(['Net Immediate', 'Net Future'], [net_immediate, net_future], color=['blue', 'green'])
    ax.axhline(0, color='black', linewidth=0.8, linestyle='--')
    ax.set_title('Net Payment Balances (Immediate vs Future)')
    ax.set_ylabel('Net Payment Balance')
    st.pyplot(fig)

    # KPI 4: Immediate vs Future In Payments
    st.subheader("KPI 4: Immediate vs Future In Payments")
    fig, ax = plt.subplots(figsize=(8, 5))
    ax.bar(['Immediate In', 'Future In'], [total_immediate_in, total_future_in], color=['blue', 'green'])
    ax.set_title('Comparison of Immediate and Future In Payments')
    ax.set_ylabel('Total Payments')
    st.pyplot(fig)

    # KPI 5: Immediate vs Future Out Payments
    st.subheader("KPI 5: Immediate vs Future Out Payments")
    fig, ax = plt.subplots(figsize=(8, 5))
    ax.bar(['Immediate Out', 'Future Out'], [total_immediate_out, total_future_out], color=['orange', 'red'])
    ax.set_title('Comparison of Immediate and Future Out Payments')
    ax.set_ylabel('Total Payments')
    st.pyplot(fig)

    # KPI 6: Top 6 Customers with Immediate In Payments
    st.subheader("KPI 6: Top 6 Customers with Immediate In Payments")
    top_customers = immediate_transactions.groupby('Customer_code')['int payment'].sum()
    top_customers = top_customers.sort_values(ascending=False).head(6)
    fig, ax = plt.subplots(figsize=(8, 8))
    top_customers.plot(
        kind='pie',
        autopct=lambda pct: f"{pct:.1f}%\\n(${'{:.0f}'.format(pct / 100.0 * top_customers.sum())})",
        startangle=90,
        colors=['#ff9999', '#66b3ff', '#99ff99', '#ffcc99', '#c2c2f0', '#ffb3e6']
    )
    ax.set_title('Top 6 Customers by Immediate In Payments')
    ax.set_ylabel('')
    st.pyplot(fig)

    # KPI 7: Immediate In Payments of Top 6 Customers by Year
    st.subheader("KPI 7: Immediate In Payments of Top 6 Customers by Year")
    top_6_customers = top_customers.index
    top_6_data = immediate_transactions[immediate_transactions['Customer_code'].isin(top_6_customers)]
    top_6_per_year = top_6_data.groupby(['Pur_Y', 'Customer_code'])['int payment'].sum().unstack()
    fig, axes = plt.subplots(1, 6, figsize=(20, 8), sharey=True)
    colors = ['skyblue', 'lightgreen', 'salmon', 'gold', 'mediumpurple', 'coral']
    for i, (customer, color) in enumerate(zip(top_6_customers, colors)):
        bars = top_6_per_year[customer].plot(kind='bar', ax=axes[i], title=f'Customer {customer}', color=color)
        axes[i].set_xlabel('Year')
        axes[i].set_ylabel('Total Payments' if i == 0 else '')
        axes[i].tick_params(axis='x', rotation=45)
        for bar in bars.patches:
            yval = bar.get_height()
            if yval > 0:
                axes[i].text(bar.get_x() + bar.get_width() / 2, yval + yval * 0.03, f'{yval:.2f}', ha='center', va='bottom')
    st.pyplot(fig)

    # KPI 8: Total Income Over the Period
    st.subheader("KPI 8: Total Income Over the Period")
    total_income_per_year = immediate_transactions.groupby('Pur_Y')['int payment'].sum()
    fig, ax = plt.subplots(figsize=(12, 6))
    colors = plt.cm.tab20(range(len(total_income_per_year)))
    bars = ax.bar(total_income_per_year.index, total_income_per_year.values, color=colors)
    ax.set_title('Total Income of the Company Over the Period')
    ax.set_xlabel('Year')
    ax.set_ylabel('Total Income')
    plt.xticks(rotation=45)
    for bar in bars:
        yval = bar.get_height()
        ax.text(bar.get_x() + bar.get_width() / 2, yval + max(yval * 0.03, 10000), f'{yval:.2f}', ha='center', va='bottom')
    st.pyplot(fig)

else:
    st.warning("Please upload a dataset to proceed.")
""")

# Step 4: Run Streamlit App with Ngrok
from pyngrok import ngrok

# Set your ngrok authtoken
!ngrok authtoken 2qgQVKfGFXtODzSemT5wp2fGIre_3yh4GWzTaTe2P8Gyuk3cz  # Replace with your actual authtoken from ngrok dashboard

# Start the Streamlit app
!streamlit run streamlit_app.py &>/dev/null&

# Expose the app via ngrok
public_url = ngrok.connect(8501)
print("Streamlit app URL:", public_url)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.4 MB/s eta 0:00:00
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Streamlit app URL: NgrokTunnel: "https://c116-34-48-188-110.ngrok-free.app" -> "http://localhost:8501"
